# Intent-Based Customer Segmentation
- Please use this notebook to complete assignment one of the data engineering take home challenge.
- Your goal is to derive a customer segmentation based on `HIGH`, `MEDIUM`, or `LOW` buying-intent only using SQL.
- Your final output should be 2-column table (`company_id`, `buying_intent`) and it should not take you longer than 1h to get there.

In [269]:
import duckdb
con = duckdb.connect("../data/db_src.duckdb")


In [262]:
all_tables = con.sql(
    """
    SHOW ALL TABLES
    """
).fetchdf()
all_tables

,database,schema,name,column_names,column_types,temporary
0,db_src,crm,_dlt_loads,"[load_id, schema_name, status, inserted_at, sc...","[VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...",False
1,db_src,crm,_dlt_pipeline_state,"[version, engine_version, pipeline_name, state...","[BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...",False
2,db_src,crm,_dlt_version,"[version, engine_version, inserted_at, schema_...","[BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...",False
3,db_src,crm,companies,"[id, address, domain, _dlt_load_id, _dlt_id]","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR]",False
4,db_src,crm,companies__contacts,"[id, first_name, last_name, email, phone, _dlt...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, ...",False
5,db_src,crm,contacts,"[company_id, id, first_name, last_name, email,...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, ...",False
6,db_src,crm,customer_buying_intent,"[company_id, buying_intent]","[VARCHAR, VARCHAR]",False
7,db_src,crm,news_publications,"[id, headline, company_website, publication_da...","[VARCHAR, VARCHAR, VARCHAR, DATE, VARCHAR, VAR...",False
8,db_src,crm,outreach_emails,"[id, to, from, subject, sent_date_utc, _dlt_lo...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, DATE, VAR...",False
9,db_src,crm,website_visits,"[id, email, event_date_utc, _dlt_load_id, _dlt...","[VARCHAR, VARCHAR, DATE, VARCHAR, VARCHAR]",False


In [83]:
import pandas as pd

In [84]:
companies = con.sql(
    """
    select * from crm.companies
    """
)# .fetchdf()

companies__contacts  = con.sql(
    """
    select * from crm.companies__contacts
    """
)# .fetchdf()

contacts = con.sql(
    """
    select * from crm.contacts
    """
)# .fetchdf()

news_publications = con.sql(
    """
    select * from crm.news_publications
    """
)# .fetchdf()

outreach_emails = con.sql(
    """
    select * from crm.outreach_emails
    """
)# .fetchdf()

website_visits = con.sql(
    """
    select * from crm.website_visits
    """
)# .fetchdf()

In [259]:
contacts_with_domain = con.sql(
    """
    SELECT c1.*, c2.domain from contacts as c1
    JOIN companies as c2 on c1.company_id = c2.id
    """
)


contacts_with_domain_website_visits = con.sql(
    """
    SELECT c.company_id , 
        c.domain,
        Count(w.email) AS email_count, 
        CASE
             WHEN Count(w.email) >= 1 THEN 1 ELSE 0
        END as Medium_Or_High
    From contacts_with_domain as c
    LEFT JOIN website_visits as w on w.email = c.email
    Group by c.company_id, c.domain
    """
)


news_publications_processed = con.sql(
    """
    Select *, 
    CASE 
        WHEN company_website LIKE 'http://www.%' THEN REGEXP_REPLACE(company_website, '^http://www\\.', '')
        WHEN company_website LIKE 'https://www.%' THEN REGEXP_REPLACE(company_website, '^https://www\\.', '')
        WHEN company_website LIKE 'http://%' THEN REGEXP_REPLACE(company_website, '^http://', '')
        WHEN company_website LIKE 'https://%' THEN REGEXP_REPLACE(company_website, '^https://', '')
        ELSE company_website 
    END AS cleaned_domain,
    CAST(JULIAN(CAST('2024-04-30' AS DATE)) - JULIAN(publication_date_utc) AS INTEGER) AS days
    from news_publications
    Order by days asc 
    """
)


ranked_news = con.sql(
    """
    WITH Ranked AS (
        SELECT *,
            ROW_NUMBER() OVER (PARTITION BY company_website ORDER BY days ASC) AS rank
        FROM news_publications_processed
    )
    SELECT * FROM Ranked WHERE rank = 1
    """
)

joined_table = con.sql(
    """
      SELECT distinct c.*, r.days,
      CASE
        WHEN c.Medium_Or_High = 1 AND r.days <= 7 THEN 'HIGH' 
        WHEN c.Medium_Or_High = 0 and r.days Between 8 and 30 THEN 'MEDIUM' 
       Else 'LOW'
      END AS buying_intent
      From contacts_with_domain_website_visits as c
    JOIN ranked_news as r ON c.domain = r.cleaned_domain
    
    """
)


ranked_join = con.sql(
    """
    WITH ranked AS (
        SELECT *,
            ROW_NUMBER() OVER (PARTITION BY company_id ORDER BY buying_intent) AS row_num
        FROM joined_table
    )
    SELECT company_id, buying_intent FROM ranked WHERE row_num = 1;
        """
)

print(ranked_join)
print(con.sql("""SELECT buying_intent, Count(*) FROM ranked_join Group by buying_intent"""))


┌──────────────────────────────────────┬───────────────┐
│              company_id              │ buying_intent │
│               varchar                │    varchar    │
├──────────────────────────────────────┼───────────────┤
│ 0a24c52e-caf5-4cfd-89b5-5719e97e9fc3 │ LOW           │
│ 8da175d5-86ff-4db9-8c4b-959a0b7414c3 │ MEDIUM        │
│ b9ffe83e-22a7-4917-8781-8a8f9ab50e5c │ LOW           │
│ eab25265-1764-4b4f-a8ad-d918d9b58f42 │ LOW           │
│ 0effc628-155e-4b94-b0c5-e52b513f99cf │ HIGH          │
│ 446fe34c-99b8-400c-a4f6-79fa5e2dd679 │ LOW           │
│ b39c5f93-3d1d-45e5-8d65-c7d9fd92d6e1 │ HIGH          │
│ c16e1a92-fa26-48ad-84cd-14a5d1ac2a3f │ LOW           │
│ ead84bf6-d467-4ae9-9a2d-28d483edc9c0 │ LOW           │
│ f67a0e28-5a3a-49da-99de-0e61e07681fd │ LOW           │
│                  ·                   │  ·            │
│                  ·                   │  ·            │
│                  ·                   │  ·            │
│ 93942bca-f331-4410-93e9-cee4c

In [261]:
# Please use the space below to express your segmentation logic in SQL and write it back into the CRM database. 
con.sql(
    """
    create or replace table crm.customer_buying_intent as (
        -- your segmentation query goes here
        Select * from ranked_join
    )
    """
)


In [271]:
con.close()

In [ ]:
# Backup: In case you get stuck above, you can run this cell to import the segmentation logic from pre-made csv file
con.sql("create table crm.customer_buying_intent as from read_csv('data/customer_segments.csv')")
